### Парсинг телефонов с Авито

In [14]:
import sys
sys.path.append('../..')


In [15]:
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from tqdm import tqdm
from webdriver_manager.firefox import GeckoDriverManager
from kafka import KafkaProducer
import psycopg2
import json

from src.parser.models import Product, Seller
from src.parser.parser import parse_avito_page
from src.parser.tools import get_ad_urls

### Константы

In [16]:
URL_TEMPLATE = "https://www.avito.ru/sankt-peterburg/telefony/mobilnye_telefony/apple-ASgBAgICAkS0wA3OqzmwwQ2I_Dc?p="
PAUSE_DURATION_SECONDS = 2
NUM_PAGES = 2

producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

def send_phone(data):
    producer.send("phone_listings", value=data)
    producer.flush()

def send_seller(data):
    producer.send("seller_listings", value=data)
    producer.flush()


# списки существующих ключей для того, чтобы лишний раз 
# не добавлять объявление/ продавца в  таблицу
conn = psycopg2.connect("dbname=mydatabase user=myuser password=mypassword host=localhost port=5432")
cursor = conn.cursor()

cursor.execute("SELECT link FROM product")
product_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT seller_id FROM seller")
seller_ids = [row[0] for row in cursor.fetchall()]

cursor.close()
conn.close()

### Запуск сервиса

In [17]:
service = Service(GeckoDriverManager().install())
# add headers
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
#     "Accept-Language": "en-US,en;q=0.9",
#     "Accept-Encoding": "gzip, deflate, br",
# }

# create a new session
# session = Session()
# session.headers.update(headers)

# dont open browser
options = webdriver.FirefoxOptions()
# options.add_argument('--headless')
driver = webdriver.Firefox(service=service, options=options)

### Максимальный номер сохраненной страницы  

In [18]:
start_page_number = 1
end_page_number = start_page_number + NUM_PAGES

print(f"Начнем парсинг со страницы номер: {start_page_number} (вкл)")
print(f"Закончим парсинг на странице номер: {end_page_number} (не вкл)")

Начнем парсинг со страницы номер: 1 (вкл)
Закончим парсинг на странице номер: 3 (не вкл)


### Парсинг

In [19]:
for page_num in range(start_page_number, end_page_number):
    
    print(f"Начали парсинг страницы # {page_num}")

    try:
        # Загрузка страницы с объявлениями
        url = URL_TEMPLATE + str(page_num)
        driver.get(url)
        time.sleep(PAUSE_DURATION_SECONDS)

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
  
        # Извлекаем ссылки на объявления на текущей странице
        links = get_ad_urls(soup)
        for link in tqdm(links):
            if link in product_ids:
                continue
            try:
                # Переход на страницу объявления
                driver.get(link)
                time.sleep(PAUSE_DURATION_SECONDS)  # Задержка для полной загрузки

                # Парсим данные на странице объявления (название, цена, фото, описание и т.д.)
                ad_data, seller_data, done_deals_data = parse_avito_page(driver=driver) # <- словарик 
                ad_data['link'] = link

                # валидация используя Pydantic
                product = Product.model_validate(ad_data)
                seller = Seller.model_validate(seller_data)

                if ad_data['link'] in product_ids:
                    continue
                
                product_ids.append(link)
                send_phone(product.model_dump())
                # проверка на отсутствие продавца в таблице
                if seller_data['seller_id'] not in seller_ids:
                    seller_ids.append(seller_data['seller_id'])
                    send_seller(seller.model_dump())

                    done_deals_list = [Product.model_validate(deal) for deal in done_deals_data]
                    for deal in done_deals_list:
                        send_phone(deal.model_dump())
            except TimeoutException:
                print(f"Ошибка: объявление {link} не загрузилось, пропускаем...")
                continue
            except Exception as e:
                print(f"Произошла ошибка при обработке объявления {link}: {e}")
                continue
    except TimeoutException:
        print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
        continue
    except Exception as e:
        print(f"Произошла ошибка на странице {page_num}: {e}")
        continue


Начали парсинг страницы # 1


100%|██████████| 50/50 [00:21<00:00,  2.33it/s]

Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_13_pro_128_gb_3294448874?context=H4sIAAAAAAAA_wE_AMD_YToyOntzOjEzOiJsb2NhbFByaW9yaXR5IjtiOjA7czoxOiJ4IjtzOjE2OiIwR3kwT3dXdXEwVzUzUTJXIjt9H0ByST8AAAA: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:197:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:706:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:538:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.getPageSource@chrome://remote/content/marionette/driver.sys.mjs:1146:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/conte

In [20]:
driver.quit()